In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=474426c95a793eb2049caffdd488c7e74d1cd762b61d5374d4a174c68709d721
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [3]:
import os
import pandas as pd
import random
import zipfile

In [4]:
!python -m wget https://filr.hs-offenburg.de/filr/public-link/file-download/0dcf8b85882ae199018870bceddf437c/5426/-2254907074900866378/test_data.zip


Saved under test_data.zip


In [5]:
!python -m wget https://zenodo.org/record/7869954/files/products_leaflets_256.zip?download=1


Saved under products_leaflets_256.zip


In [6]:
path_to_zip_file = 'products_leaflets_256.zip'
directory_to_extract_to = '.'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [7]:
path_to_zip_file = 'test_data.zip'
directory_to_extract_to = '.'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [8]:
import os
import numpy as np
from PIL import Image



# Verzeichnis mit den Trainingsdaten
train_directory = 'products_leaflets_256/train'

# Bildgröße und Anzahl der Klassen
image_size = (224, 224)
num_classes = len(os.listdir(train_directory))

# Liste für Bilder und Labels
images = []
labels = []

# Durchlaufe das Trainingsverzeichnis und lade die Bilder
for root, dirs, files in os.walk(train_directory):
    for file in files:
        if file.endswith('.jpg'):
            image_path = os.path.join(root, file)
            image = Image.open(image_path).convert('RGB')
            image = image.resize(image_size)
            image_array = np.array(image)
            images.append(image_array)
            labels.append(int(os.path.basename(root)))

# Konvertiere die Listen in Numpy-Arrays
images = np.array(images)
labels = np.array(labels)




In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

#ResNet50 with defaul parameters
model = ResNet50()
model2 = model.compile()


# Trainieren des Modells mit 10 durchläufen
model.fit(images, labels, batch_size=32, epochs=10, validation_split=0.1)

In [ ]:
# Liste für Bildnamen und Vorhersagen
image_names = []
label_predictions = []

# Durchlaufe das Testverzeichnis und lade die Bilder
for root, dirs, files in os.walk(test_directory):
    for file in files:
        if file.endswith('.jpg'):
            image_path = os.path.join(root, file)
            image = Image.open(image_path).convert('RGB')
            image = image.resize(image_size)
            image_array = np.array(image)
            image_names.append(file)
            label_prediction = model.predict(np.expand_dims(image_array, axis=0))
            label_predictions.append(np.argmax(label_prediction))

# Erstelle den DataFrame und speichere die Vorhersagen in einer CSV-Datei
df = pd.DataFrame(columns=['image_id', 'label'])
df['image_id'] = image_names
df['label'] = label_predictions
df.to_csv('submission.csv', index=False)

# Neuer Abschnitt